In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow_datasets as tfds

In [2]:
def preprocess_image(img, lbl):
    resized_img = tf.image.resize(img, (224, 224))
    return resized_img, lbl


In [3]:
(train_data, val_data, test_data), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete7QW3R0/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [4]:
# Apply preprocessing to each dataset
train_data = train_data.map(preprocess_image)
val_data = val_data.map(preprocess_image)
test_data = test_data.map(preprocess_image)

In [5]:
BUFFER_SIZE = 1000
train_ds = train_data.shuffle(BUFFER_SIZE).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_data.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_data.batch(32)

In [6]:
# Use ResNet50 model without the top layer and with imagenet weights
resnet50_base = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Make sure the base model is not trainable
resnet50_base.trainable = False


94765736/94765736 [==============================] - 5s 0us/step


In [7]:
# Construct the full model
x = resnet50_base.output
x = GlobalAveragePooling2D()(x)
output_layer = Dense(1, activation='sigmoid')(x)

In [8]:
# Define the model
cats_dogs_model = Model(inputs=resnet50_base.input, outputs=output_layer)

# Compile
cats_dogs_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Training
history = cats_dogs_model.fit(train_ds, validation_data=val_ds, epochs=5)


Epoch 1/5
582/582 [==============================] - 81s 117ms/step - loss: 0.0839 - accuracy: 0.9668 - val_loss: 0.0702 - val_accuracy: 0.9764
Epoch 2/5
582/582 [==============================] - 69s 116ms/step - loss: 0.0497 - accuracy: 0.9810 - val_loss: 0.0534 - val_accuracy: 0.9819
Epoch 3/5
582/582 [==============================] - 69s 117ms/step - loss: 0.0426 - accuracy: 0.9841 - val_loss: 0.0562 - val_accuracy: 0.9807
Epoch 4/5
582/582 [==============================] - 69s 117ms/step - loss: 0.0388 - accuracy: 0.9861 - val_loss: 0.0538 - val_accuracy: 0.9819
Epoch 5/5
582/582 [==============================] - 69s 116ms/step - loss: 0.0352 - accuracy: 0.9865 - val_loss: 0.0586 - val_accuracy: 0.9794


In [9]:
eval_loss, eval_accuracy = cats_dogs_model.evaluate(test_ds)
print(f"\nEvaluation accuracy: {eval_accuracy * 100:.2f}%")

73/73 [==============================] - 7s 99ms/step - loss: 0.0510 - accuracy: 0.9802

Evaluation accuracy: 98.02%
